In [9]:
import smtplib
from email.mime.text import MIMEText
from newspaper import Article
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import re
from dotenv import load_dotenv
import os

/Users/tusharrohilla/Desktop/AI-news-headline-wishperer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 1: Scrape the news article

In [2]:
def scrape_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.text

Step 2: Generate Summary

In [10]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") #For Apple Silicon Macs
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # For windows users

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
model = model.to(device)

In [13]:
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=-1)

Device set to use mps:0


In [ ]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip() #Removes the white space
    return text

def summarize_text(text):
    cleaned_text = clean_text(text)
    summary = summarizer(cleaned_text, max_length=150, min_length=50, do_sample=False)[0]['summary_text']
    return summary

Step 3: Send generated summary to the email

In [14]:
load_dotenv()  # Load variables from .env

sender_email = os.getenv("sender_email")
sender_password = os.getenv("sender_password")

In [12]:
# Function to send email
def send_email(summary, recipient_email, sender_email, sender_password):
    try:
        msg = MIMEText(summary)
        msg['Subject'] = 'Article Summary'
        msg['From'] = sender_email
        msg['To'] = recipient_email

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
        return True
    except Exception as e:
        return f"Error sending email: {str(e)}"

In [18]:
url = "https://timesofindia.indiatimes.com/city/srinagar/jaw-broken-spine-injured-spicejet-staff-attacked-by-passenger-at-srinagar-airport/articleshow/123073510.cms"
#Step 1 : Scrape the article
article_text = scrape_article(url)

#Step 2: Generate Summary
summary = summarize_text(article_text)

#Step 3: Send generated summary to the email
recipient_email = "tusharrohilla70@gmail.com"
email_sent = send_email(summary, recipient_email, sender_email, sender_password)